In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
# Collect 50% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_50, _, y_50 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.5, stratify=y_label_0_1, random_state=42)

In [6]:
X_50.shape

(398899, 56)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb50 = XGBClassifier()
xgb50.fit(X_50, y_50)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf50 = RandomForestClassifier()
rf50.fit(X_50, y_50)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt50 = DecisionTreeClassifier()
dt50.fit(X_50, y_50)

xgb
RF
DT


DecisionTreeClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude2Attack/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude2Attack/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [   179    295   2019   1868     22    141      1     69 109090    226
     61]
(179, 56) (179,)
Save 0 to baseline_Def1.npy
(295, 56) (295,)
Save 1 to baseline_Def2.npy
(2019, 56) (2019,)
Save 2 to baseline_Def3.npy
(1868, 56) (1868,)
Save 3 to baseline_Def4.npy
(22, 56) (22,)
Save 4 to baseline_Def5.npy
(141, 56) (141,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(69, 56) (69,)
Save 7 to baseline_Def8.npy
(109090, 56) (109090,)
Save 8 to baseline_Def9.npy
(226, 56) (226,)
Save 9 to baseline_Def10.npy
(61, 56) (61,)
Save 10 to baseline_Def11.npy
Execution Time: 0.477026 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [101918    126   1087    891    410 144447      1    260  92685     64
     24]
(101918, 56) (101918,)
Save 0 to BIM_Def1.npy
(126, 56) (126,)
Save 1 to BIM_Def2.npy
(1087, 56) (1087,)
Save 2 to BIM_Def3.npy
(891, 56) (891,)
Save 3 to BIM_Def4.npy
(410, 56) (410,)
Save 4 to BIM_Def5.npy
(144447, 56

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [   373    307   2169   1856     52    801      1    191 107932    209
     80]
(373, 56) (373,)
Save 0 to baseline_Def1.npy
(307, 56) (307,)
Save 1 to baseline_Def2.npy
(2169, 56) (2169,)
Save 2 to baseline_Def3.npy
(1856, 56) (1856,)
Save 3 to baseline_Def4.npy
(52, 56) (52,)
Save 4 to baseline_Def5.npy
(801, 56) (801,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(191, 56) (191,)
Save 7 to baseline_Def8.npy
(107932, 56) (107932,)
Save 8 to baseline_Def9.npy
(209, 56) (209,)
Save 9 to baseline_Def10.npy
(80, 56) (80,)
Save 10 to baseline_Def11.npy
Execution Time: 1.564244 seconds
(341913,)
[0 2 3 4 5 7 8 9] [ 60411    705    159      1 167753      8 112871      5]
(60411, 56) (60411,)
Save 0 to BIM_Def1.npy
(705, 56) (705,)
Save 2 to BIM_Def3.npy
(159, 56) (159,)
Save 3 to BIM_Def4.npy
(1, 56) (1,)
Save 4 to BIM_Def5.npy
(167753, 56) (167753,)
Save 5 to BIM_Def6.npy
(8, 56) (8,)
Save 7 to BIM_Def8.npy
(112871, 56) (

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude2Attack/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude2Attack/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [   521    508   2681   2180     84   1239      1    383 105894    321
    159]
(521, 56) (521,)
Save 0 to baseline_Def1.npy
(508, 56) (508,)
Save 1 to baseline_Def2.npy
(2681, 56) (2681,)
Save 2 to baseline_Def3.npy
(2180, 56) (2180,)
Save 3 to baseline_Def4.npy
(84, 56) (84,)
Save 4 to baseline_Def5.npy
(1239, 56) (1239,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(383, 56) (383,)
Save 7 to baseline_Def8.npy
(105894, 56) (105894,)
Save 8 to baseline_Def9.npy
(321, 56) (321,)
Save 9 to baseline_Def10.npy
(159, 56) (159,)
Save 10 to baseline_Def11.npy
Execution Time: 0.532345 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [ 84085   2226   6681   5102   1416 154306   2142   8073  77007    605
    270]
(84085, 56) (84085,)
Save 0 to BIM_Def1.npy
(2226, 56) (2226,)
Save 1 to BIM_Def2.npy
(6681, 56) (6681,)
Save 2 to BIM_Def3.npy
(5102, 56) (5102,)
Save 3 to BIM_Def4.npy
(1416, 56) (1416,)
Save 4 to BIM_Def5.npy
(

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}50")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_Exclude2Attack/UNSW_Input50/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input50_Exclude2Attack.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB50,179,295,2019,1868,22,141,1,69,109090,226,61,113971
1,XGB50,101918,126,1087,891,410,144447,1,260,92685,64,24,341913
2,XGB50,72287,45,929,923,207,173792,1,1953,91709,32,35,341913
3,XGB50,101918,126,1087,891,410,144447,1,260,92685,64,24,341913
4,XGB50,74076,634,2248,344,211,243782,380,17964,1906,242,126,341913
5,XGB50,88511,771,4174,4216,410,148039,1,307,94761,556,167,341913
6,XGB50,582,848,6051,5123,66,491,4,205,327759,615,169,341913
7,XGB50,42641,31,601,1538,25,1905,2,10,295083,14,63,341913
8,XGB50,104483,494,558,889,350,134740,1,119,100229,37,13,341913
9,XGB50,88024,59,1151,909,544,158814,1,137,92198,34,42,341913
